## Advanced Lane Finding Project

The goal of this project is to identify and mark the driving lanes in the provided input images.  

The steps of this project are the following:

* Setups

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; S1.  Compute the camera calibration matrix and distortion coefficients given a set of chessboard images  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; S2.  Setup helper functions for color transform / thresholding  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; S3.  Obtain warp matrix for perspective transform  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; S4.  Setup helper functions for lane recognition  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; S5.  Setup helper functions for curvature calculation  

* Applications  

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A1. Apply a distortion correction to raw images  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A2. Use color transforms, gradients, etc., to create a thresholded binary image  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A3. Apply a perspective transform to rectify binary image ("birds-eye view")  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A4. Detect lane pixels and fit to find the lane boundary  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A5. Determine the curvature of the lane and vehicle position with respect to center  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A6. Warp the detected lane boundaries back onto the original image  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; A7. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position

## S1. Get camera calibration matrix

In [1]:
# Library list
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
%matplotlib qt

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import imageio
imageio.plugins.ffmpeg.download()

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

## S2. Setup helper functions for color transform / thresholding

In [ ]:
# Note: format of the original image is BGR, NOT RGB, since the images are imported by OpenCV
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # 1. Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 2. Take the derivative in each direction and take absolute value
    if orient == 'x':
        absSobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    if orient == 'y':
        absSobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    # 3. Scale to 8-bit and convert to uint8
    scaledSobel = np.uint8(255*absSobel/np.max(absSobel))
    # 4. Create a mask of 1s based on threshold
    grad_binary = np.zeros_like(scaledSobel)
    grad_binary[(scaledSobel >= thresh[0]) & (scaledSobel <= thresh[1])] = 1
    # Return result
    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # 1. Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 2. Take the derivative in both directions and take absolute values
    absX = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    absY = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    # 3. Take the magnitude of derivatives
    absXY = np.sqrt(absX**2 + absY**2)
    # 4. Scale to 8-bit and convert to uint8
    scaledXY = np.uint8(255*absXY/np.max(absXY))
    # 5. Create a mask of 1s based on threshold
    mag_binary = np.zeros_like(scaledXY)
    mag_binary[(scaledXY >= mag_thresh[0])&(scaledXY <= mag_thresh[1])] = 1 
    # Return result
    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # 1. Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 2. Take the derivative in both directions and take absolute values
    absX = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel))
    absY = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel))
    # 3. Take the direction of derivatives
    absDir = np.arctan2(absY, absX)
    # 4. Create a mask of 1s based on threshold
    dir_binary = np.zeros_like(absDir)
    dir_binary[(absDir >= thresh[0]) & (absDir <= thresh[1])] = 1
    # Return result
    return dir_binary

def B_thresh(img, thresh=(0,255)):
    unicolor = img[:,:,0]
    output = np.zeros_like(unicolor)
    output[(unicolor >= thresh[0]) & (unicolor <= thresh[1])] = 1
    return output

def G_thresh(img, thresh=(0,255)):
    unicolor = img[:,:,1]
    output = np.zeros_like(unicolor)
    output[(unicolor >= thresh[0]) & (unicolor <= thresh[1])] = 1
    return output

def R_thresh(img, thresh=(0,255)):
    unicolor = img[:,:,2]
    output = np.zeros_like(unicolor)
    output[(unicolor >= thresh[0]) & (unicolor <= thresh[1])] = 1
    return output

def H_thresh(img, thresh=(0,255), dynamicMode = 0):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    unicolor = img[:,:,0]
    
    if dynamicMode == 1:
        temp = np.amax(unicolor)
        thresh = (temp*thresh[0], temp)
    
    output = np.zeros_like(unicolor)
    output[(unicolor >= thresh[0]) & (unicolor <= thresh[1])] = 1
    return output

def L_thresh(img, thresh=(0,255)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    unicolor = img[:,:,1]
    output = np.zeros_like(unicolor)
    output[(unicolor >= thresh[0]) & (unicolor <= thresh[1])] = 1
    return output

def S_thresh(img, thresh=(0,255), dynamicMode = 0):
    # When dinamicMode = 1, thresh = (relativeStrength, None)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    unicolor = img[:,:,2]
    
    if dynamicMode == 1:
        temp = np.amax(unicolor)
        thresh = (temp*thresh[0], temp)
    
    output = np.zeros_like(unicolor)
    output[(unicolor >= thresh[0]) & (unicolor <= thresh[1])] = 1
    return output

def regionOfInterest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [ ]:
def colorTransform(undistorted):
    HLSS = S_thresh(undistorted, thresh=(0.5, 0), dynamicMode = 1) # originally: (150,255)
    HLSH = H_thresh(undistorted, thresh=(70, 100), dynamicMode = 0)
    BGRR = R_thresh(undistorted, thresh=(200,255))

    combined = np.zeros_like(HLSS)
    combined[(HLSH == 1) & (HLSS == 1) | ((BGRR == 1))] = 1
    
    return combined

## S3. Obtain warp matrix for perspective transform

In [5]:
# 4 points on warpSrc are manually picked based on observation
warpSrc = np.float32([[430.5, 566], [867.7, 566],[1052, 682],[269, 682]])
warpDst = np.float32([[269, 566], [1052, 566], [1052, 682], [269, 682]])

# Warp matrix and its inverse
M = cv2.getPerspectiveTransform(warpSrc, warpDst)
Minv = cv2.getPerspectiveTransform(warpDst, warpSrc)    

## S4.  Setup helper functions for lane recognition 

In [6]:
def slidingWindowsHelper(binary_warped, _base, nwindows, bias, lr):
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    _current = _base

    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    _lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low =  (window)*window_height
        win_y_high = (window+1)*window_height
        win_x_low = _current - margin
        win_x_high = _current + margin

        # Draw the windows on the visualization image
        #cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_x_low) & (nonzerox < win_x_high)).nonzero()[0]
        
        # Append these indices to the lists
        _lane_inds.append(good_inds)
        
        if bias == 1:
            base_y_low = 0
            base_y_high = 80
            if lr == 'L':
                base_x_low = 230
                base_x_high = 330
            elif lr == 'R':
                base_x_low = 1070
                base_x_high = 1170
            good_inds = ((nonzeroy >= base_y_low) & (nonzeroy < base_y_high) & (nonzerox >= base_x_low) & (nonzerox < base_x_high)).nonzero()[0]
        
            # Append these indices to the lists
            _lane_inds.append(good_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_inds) > minpix:
            _current = np.int(np.mean(nonzerox[good_inds]))

    # Concatenate the arrays of indices
    _lane_inds = np.concatenate(_lane_inds)
    
    return _lane_inds, nonzerox, nonzeroy

In [7]:
def slidingWindows(left_binary_warped, right_binary_warped, leftLine, rightLine, nwindows = 9, confidenceCheck = 1, polynomialCheck =1, segmentCheck = 1, debugMode = 0):
    
    # Take a histogram of the bottom half of the image
    leftHistogram = np.sum(left_binary_warped[:left_binary_warped.shape[0]//2,:], axis=0)
    rightHistogram = np.sum(right_binary_warped[:right_binary_warped.shape[0]//2,:], axis=0)
    
    leftx_base = np.argmax(leftHistogram)
    rightx_base = np.argmax(rightHistogram)
    
    left_lane_inds, leftNonzerox, leftNonzeroy = slidingWindowsHelper(left_binary_warped, leftx_base, nwindows, bias = 0, lr = 'L')
    right_lane_inds, rightNonzerox, rightNonzeroy = slidingWindowsHelper(right_binary_warped, rightx_base, nwindows, bias = 0, lr = 'R')
    
    # Extract left and right line pixel positions
    leftLine.addAllX(leftNonzerox[left_lane_inds])
    leftLine.addAllY(leftNonzeroy[left_lane_inds])
    rightLine.addAllX(rightNonzerox[right_lane_inds])
    rightLine.addAllY(rightNonzeroy[right_lane_inds])
    
    # Initialize variables
    leftLine.current_fit = []
    rightLine.current_fit = []
    leftLine.best_fit = None
    rightLine.best_fit = None
    leftLine.diffs = [0,0,0]
    rightLine.diffs = [0,0,0]

    # Fit a second order polynomial to each
    leftLine.runningWindow(np.polyfit(leftLine.ally, leftLine.allx, 2))
    rightLine.runningWindow(np.polyfit(rightLine.ally, rightLine.allx, 2))
    
    # Check how confident each polynomial fit is
    if confidenceCheck == 1:
        leftLine.noLineFound = False
        rightLine.noLineFound = False
        if len(left_lane_inds) < 500:
            leftLine.lowConfidenceFlag += 1
        elif leftLine.lowConfidenceFlag >= 0:
            leftLine.lowConfidenceFlag -= 1
        if len(right_lane_inds) < 500:
            rightLine.lowConfidenceFlag += 1
        elif rightLine.lowConfidenceFlag >= 0:
            rightLine.lowConfidenceFlag -= 1
        
        if (leftLine.lowConfidenceFlag > 2 and rightLine.lowConfidenceFlag <= 2):
            if len(leftLine.current_fit)>1:
                leftLine.current_fit = np.delete(leftLine.current_fit, -1, 0)
                new = [rightLine.best_fit[0]-0.3*np.absolute(rightLine.best_fit[0]), rightLine.best_fit[1], leftLine.best_fit[2]]
                if len(leftLine.current_fit)>1:
                    leftLine.current_fit = np.vstack((leftLine.current_fit, new))
                else:
                    leftLine.current_fit = [new]
            leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
            leftLine.lowConfidenceFlag -= 1
        elif (leftLine.lowConfidenceFlag <= 2 and rightLine.lowConfidenceFlag > 2):
            if len(rightLine.current_fit)>1:
                rightLine.current_fit = np.delete(rightLine.current_fit, -1, 0)
                new = [leftLine.best_fit[0]+0.3*np.absolute(leftLine.best_fit[0]), leftLine.current_fit[1], rightLine.best_fit[2]]
                if len(rightLine.current_fit)>1:
                    rightLine.current_fit = np.vstack((rightLine.current_fit, new))
                else:
                    rightLine.current_fit = [new]
            rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
            rightLine.lowConfidenceFlag -= 1
        elif (leftLine.lowConfidenceFlag > 2 and rightLine.lowConfidenceFlag > 2):
            leftLine.noLineFound = True
            leftLine.lowConfidenceFlag -= 1
            rightLine.noLineFound = True
            rightLine.lowConfidenceFlag -= 1
        
        leftLine.confidence = len(left_lane_inds)
        rightLine.confidence = len(right_lane_inds)

    
    # Check if the segment of fitted line sits in resonable range
    if segmentCheck == 1:
        LL = 170
        LR = 450
        RL = 950
        RR = 1130
        
        if (leftLine.best_fit[2] > LL and leftLine.best_fit[2] < LR) != 1:
            if (rightLine.best_fit[2] > RL and rightLine.best_fit[2] < RR) != 1: # Left & Right segment out of range
                if len(leftLine.current_fit) > 1:
                    leftLine.current_fit = np.delete(leftLine.current_fit, -1, 0)
                leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
                if len(rightLine.current_fit) > 1:
                    rightLine.current_fit = np.delete(rightLine.current_fit, -1, 0)
                rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
                
            else: # Only left segment out of range
                if len(leftLine.current_fit) > 1:
                    leftLine.current_fit = np.delete(leftLine.current_fit, -1, 0)
                    new = [rightLine.best_fit[0], rightLine.best_fit[1], leftLine.segment]
                    leftLine.current_fit = np.vstack((leftLine.current_fit, new))
                    leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
                elif len(leftLine.current_fit) == 1:
                    if leftLine.segment == None:
                        new = [rightLine.best_fit[0], rightLine.best_fit[1], leftLine.current_fit[-1][2]]
                    else:
                        new = [rightLine.best_fit[0], rightLine.best_fit[1], leftLine.segment]
                    leftLine.current_fit = [new]
                    leftLine.best_fit = new
                    
        else:
            if (rightLine.best_fit[2] > RL and rightLine.best_fit[2] < RR) != 1: # Only right segment out of range
                if len(rightLine.current_fit) > 1:
                    rightLine.current_fit = np.delete(rightLine.current_fit, -1, 0)
                    new = [leftLine.best_fit[0], leftLine.best_fit[1], rightLine.segment]
                    rightLine.current_fit = np.vstack((rightLine.current_fit, new))
                    rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
                elif len(rightLine.current_fit) == 1:
                    if rightLine.segment == None:
                        new = [leftLine.best_fit[0], leftLine.best_fit[1], rightLine.current_fit[-1][2]]
                    else:
                        new = [leftLine.best_fit[0], leftLine.best_fit[1], rightLine.segment]
                    rightLine.current_fit = [new]
                    rightLine.best_fit = new
            
        # Store segment
        leftLine.segment = leftLine.best_fit[2]
        rightLine.segment = rightLine.best_fit[2]
        
    # Check if both lines have similar curvature, as lane lines supposed to be near parallel
    if polynomialCheck == 1:
        # Check if previous number is available
        if (leftLine.polyCF != None and rightLine.polyCF != None):
            if (np.absolute(leftLine.best_fit[0] - rightLine.best_fit[0]) / min(np.absolute(leftLine.best_fit[0]), np.absolute(rightLine.best_fit[0])) > 1):
                # If left is out of range
                if (np.absolute(leftLine.polyCF-leftLine.best_fit[0]) > np.absolute(rightLine.polyCF-rightLine.best_fit[0])):
                    leftLine.current_fit[-1][0] = rightLine.best_fit[0]
                    leftLine.current_fit[-1][1] = rightLine.best_fit[1]
                    if (len(leftLine.current_fit) == 1):
                        leftLine.best_fit = leftLine.current_fit[-1]
                    else:
                        leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
                        
                # If right is out of range
                else:
                    rightLine.current_fit[-1][0] = leftLine.best_fit[0]
                    rightLine.current_fit[-1][1] = leftLine.best_fit[1]
                    if (len(rightLine.current_fit) == 1):
                        rightLine.best_fit = rightLine.current_fit[-1]
                    else:
                        rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
        
        leftLine.polyCF = leftLine.best_fit[0]
        rightLine.polyCF = rightLine.best_fit[0]
            
    
    # Generate x and y values for plotting
    leftLine.ploty = np.linspace(0, left_binary_warped.shape[0]-1, left_binary_warped.shape[0] )
    rightLine.ploty = np.linspace(0, right_binary_warped.shape[0]-1, right_binary_warped.shape[0] )
    leftLine.current_fitx = leftLine.best_fit[0]*leftLine.ploty**2 + leftLine.best_fit[1]*leftLine.ploty + leftLine.best_fit[2]
    rightLine.current_fitx = rightLine.best_fit[0]*rightLine.ploty**2 + rightLine.best_fit[1]*rightLine.ploty + rightLine.best_fit[2]
    
    leftLine.detected = True
    rightLine.detected = True

In [8]:
def knownLines(left_binary_warped, right_binary_warped, leftLine, rightLine, confidenceCheck = 1, polynomialCheck = 1, segmentCheck = 1, debugMode = 0):
    
    # from the next frame of video (also called "binary_warped")
    # It's now much easier to find line pixels!
    leftNonzero = left_binary_warped.nonzero()
    leftNonzeroy = np.array(leftNonzero[0])
    leftNonzerox = np.array(leftNonzero[1])
    rightNonzero = right_binary_warped.nonzero()
    rightNonzeroy = np.array(rightNonzero[0])
    rightNonzerox = np.array(rightNonzero[1])
    margin = 100
    left_lane_inds = ((leftNonzerox > (leftLine.best_fit[0]*(leftNonzeroy**2) + leftLine.best_fit[1]*leftNonzeroy + leftLine.best_fit[2] - margin)) & (leftNonzerox < (leftLine.best_fit[0]*(leftNonzeroy**2) + leftLine.best_fit[1]*leftNonzeroy + leftLine.best_fit[2] + margin))) 
    right_lane_inds = ((rightNonzerox > (rightLine.best_fit[0]*(rightNonzeroy**2) + rightLine.best_fit[1]*rightNonzeroy + rightLine.best_fit[2] - margin)) & (rightNonzerox < (rightLine.best_fit[0]*(rightNonzeroy**2) + rightLine.best_fit[1]*rightNonzeroy + rightLine.best_fit[2] + margin)))  

    leftLine.confidence = len(left_lane_inds)
    rightLine.confidence = len(right_lane_inds)
    
    # Again, extract left and right line pixel positions
    leftLine.addAllX(leftNonzerox[left_lane_inds])
    leftLine.addAllY(leftNonzeroy[left_lane_inds])
    rightLine.addAllX(rightNonzerox[right_lane_inds])
    rightLine.addAllY(rightNonzeroy[right_lane_inds])
    
    # Fit a second order polynomial to each
    leftLine.runningWindow(np.polyfit(leftLine.ally, leftLine.allx, 2))
    rightLine.runningWindow(np.polyfit(rightLine.ally, rightLine.allx, 2))
    
    # Check how confident each polynomial fit is
    if confidenceCheck == 1:
        leftLine.noLineFound = False
        rightLine.noLineFound = False
        if len(left_lane_inds) < 500:
            leftLine.lowConfidenceFlag += 1
        elif leftLine.lowConfidenceFlag >= 0:
            leftLine.lowConfidenceFlag -= 1
        if len(right_lane_inds) < 500:
            rightLine.lowConfidenceFlag += 1
        elif rightLine.lowConfidenceFlag >= 0:
            rightLine.lowConfidenceFlag -= 1
        
        if (leftLine.lowConfidenceFlag > 2 and rightLine.lowConfidenceFlag <= 2):
            if len(leftLine.current_fit)>1:
                leftLine.current_fit = np.delete(leftLine.current_fit, -1, 0)
                new = [rightLine.best_fit[0]-0.3*np.absolute(rightLine.best_fit[0]), rightLine.best_fit[1], leftLine.best_fit[2]]
                if len(leftLine.current_fit)>1:
                    leftLine.current_fit = np.vstack((leftLine.current_fit, new))
                else:
                    leftLine.current_fit = [new]
            leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
            leftLine.lowConfidenceFlag -= 1
        elif (leftLine.lowConfidenceFlag <= 2 and rightLine.lowConfidenceFlag > 2):
            if len(rightLine.current_fit)>1:
                rightLine.current_fit = np.delete(rightLine.current_fit, -1, 0)
                new = [leftLine.best_fit[0]+0.3*np.absolute(leftLine.best_fit[0]), leftLine.best_fit[1], rightLine.best_fit[2]]
                if len(rightLine.current_fit)>1:
                    rightLine.current_fit = np.vstack((rightLine.current_fit, new))
                else:
                    rightLine.current_fit = [new]
            rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
            rightLine.lowConfidenceFlag -= 1
        elif (leftLine.lowConfidenceFlag > 2 and rightLine.lowConfidenceFlag > 2):
            leftLine.noLineFound = True
            leftLine.lowConfidenceFlag -= 1
            rightLine.noLineFound = True
            rightLine.lowConfidenceFlag -= 1
        
        leftLine.confidence = len(left_lane_inds)
        rightLine.confidence = len(right_lane_inds)
            
    
    # Check if the segment of fitted line sits in resonable range
    if segmentCheck == 1:
        LL = 170
        LR = 450
        RL = 950
        RR = 1130
    
        if (leftLine.best_fit[2] > LL and leftLine.best_fit[2] < LR) != 1:
            if (rightLine.best_fit[2] > RL and rightLine.best_fit[2] < RR) != 1: # Left & Right segment out of range
                if len(leftLine.current_fit) > 1:
                    leftLine.current_fit = np.delete(leftLine.current_fit, -1, 0)
                leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
                if len(rightLine.current_fit) > 1:
                    rightLine.current_fit = np.delete(rightLine.current_fit, -1, 0)
                rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
                
            else: # Only left segment out of range
                if len(leftLine.current_fit) > 1:
                    leftLine.current_fit = np.delete(leftLine.current_fit, -1, 0)
                    new = [rightLine.best_fit[0], rightLine.best_fit[1], leftLine.segment]
                    leftLine.current_fit = np.vstack((leftLine.current_fit, new))
                    leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
                elif len(leftLine.current_fit) == 1:
                    if leftLine.segment == None:
                        new = [rightLine.best_fit[0], rightLine.best_fit[1], leftLine.current_fit[-1][2]]
                    else:
                        new = [rightLine.best_fit[0], rightLine.best_fit[1], leftLine.segment]
                    leftLine.current_fit = [new]
                    leftLine.best_fit = new
                    
        else:
            if (rightLine.best_fit[2] > RL and rightLine.best_fit[2] < RR) != 1: # Only right segment out of range
                if len(rightLine.current_fit) > 1:
                    rightLine.current_fit = np.delete(rightLine.current_fit, -1, 0)
                    new = [leftLine.best_fit[0], leftLine.best_fit[1], rightLine.segment]
                    rightLine.current_fit = np.vstack((rightLine.current_fit, new))
                    rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
                elif len(rightLine.current_fit) == 1:
                    if rightLine.segment == None:
                        new = [leftLine.best_fit[0], leftLine.best_fit[1], rightLine.current_fit[-1][2]]
                    else:
                        new = [leftLine.best_fit[0], leftLine.best_fit[1], rightLine.segment]
                    rightLine.current_fit = [new]
                    rightLine.best_fit = new
    
        # Store segment
        leftLine.segment = leftLine.best_fit[2]
        rightLine.segment = rightLine.best_fit[2]
        
    # Check if both lines have similar curvature, as lane lines supposed to be near parallel
    if polynomialCheck == 1:
        # Check if previous number is available
        if (leftLine.polyCF != None and rightLine.polyCF != None):
            if (np.absolute(leftLine.best_fit[0] - rightLine.best_fit[0]) / min(np.absolute(leftLine.best_fit[0]), np.absolute(rightLine.best_fit[0])) > 1):
                # If left is out of range
                if (np.absolute(leftLine.polyCF-leftLine.best_fit[0]) > np.absolute(rightLine.polyCF-rightLine.best_fit[0])):
                    leftLine.current_fit[-1][0] = rightLine.best_fit[0]
                    leftLine.current_fit[-1][1] = rightLine.best_fit[1]
                    if (len(leftLine.current_fit) == 1):
                        leftLine.best_fit = leftLine.current_fit[-1]
                    else:
                        leftLine.best_fit = np.mean(leftLine.current_fit, axis = 0)
                        
                # If right is out of range
                else:
                    rightLine.current_fit[-1][0] = leftLine.best_fit[0]
                    rightLine.current_fit[-1][1] = leftLine.best_fit[1]
                    if (len(rightLine.current_fit) == 1):
                        rightLine.best_fit = rightLine.current_fit[-1]
                    else:
                        rightLine.best_fit = np.mean(rightLine.current_fit, axis = 0)
        
        leftLine.polyCF = leftLine.best_fit[0]
        rightLine.polyCF = rightLine.best_fit[0]
    
    # Generate x and y values for plotting
    leftLine.current_fitx = leftLine.best_fit[0]*leftLine.ploty**2 + leftLine.best_fit[1]*leftLine.ploty + leftLine.best_fit[2]
    rightLine.current_fitx = rightLine.best_fit[0]*rightLine.ploty**2 + rightLine.best_fit[1]*rightLine.ploty + rightLine.best_fit[2]

## S5.  Setup helper functions for curvature calculation

In [9]:
def curvatureCalc(leftLine, rightLine):
    # Local variables
    leftx = leftLine.allx#[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightLine.allx#[::-1]  # Reverse to match top-to-bottom in y
    
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    left_fit = np.polyfit(leftLine.ally*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit = np.polyfit(rightLine.ally*ym_per_pix, rightx*xm_per_pix, 2)
    leftLine.offcenter = ((leftLine.current_fitx[0] + rightLine.current_fitx[0])/2 - 700) * xm_per_pix
    rightLine.offcenter = leftLine.offcenter
    y_eval = np.max(leftLine.ploty)
    leftLine.CR = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    rightLine.CR = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

In [10]:
def warpBackLines(left_binary_warped, leftLine, rightLine, Minv):
    warp_zero = np.zeros_like(left_binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([leftLine.current_fitx, leftLine.ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([rightLine.current_fitx, rightLine.ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    color_warp = cv2.flip(color_warp, 0)
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, img_size) 
    return newwarp    

### A1-7. Create the complete filter

In [11]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.current_fitx = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        #store segment value of best fit parameter from previous iteration
        self.segment = None
        #store polynomial coefficient of best fit parameter from previous iteration
        self.polyCF = None
        #polynomial coefficients for the most recent fit
        self.current_fit = []
        #confidence of last fitted polynomial
        self.confidence = None
        #erro counter when confidence is low
        self.lowConfidenceFlag = 0
        #if no good line is found
        self.noLineFound = False
        #radius of curvature of the line in some units
        self.CR = None 
        #distance in meters of vehicle center from the line
        self.offcenter = None 
        #difference in fit coefficients between last and new fits
        self.diffs = [0,0,0]
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #just y-axis for plotting (linspace)
        self.ploty = None
        #count how many times in a row new line did not match
        self.errCount = 0
        
    def runningWindow(self, newVal, windowSize = 5):
        if (len(self.current_fit) == 0):
            self.current_fit = [np.asarray(newVal)]
            self.best_fit = np.asarray(newVal)
        else:
            self.diffs[0] = np.absolute(newVal[0] - self.best_fit[0])/np.absolute(self.best_fit[0]) # Percentile
            self.diffs[1] = np.absolute(newVal[1] - self.best_fit[1])/np.absolute(self.best_fit[1]) # Percentile
            self.diffs[2] = np.absolute(newVal[2] - self.best_fit[2]) # Fixed value
            if ((len(self.current_fit) <= 2) or (self.diffs[0] < 0.3 and self.diffs[1] < 0.3 and self.diffs[2] < 50)):
                if len(self.current_fit) > windowSize:
                    self.current_fit = np.delete(self.current_fit, 0, 0)
                self.current_fit = np.vstack((self.current_fit, newVal))
                self.best_fit = np.mean(self.current_fit, axis = 0)
                if self.errCount <= 0:
                    self.errCount = 0
                else:
                    self.errCount -= 1
                    
            else:
                self.errCount += 1
                if self.errCount >= 4:
                    self.detected = False
                    self.errCount = 0
        
    # To handle the case when there is no new line found
    def addAllX(self, new):
        if len(new) != 0:
            self.allx = new
                
    def addAllY(self,new):
        if len(new) != 0:
            self.ally = new


In [12]:
def advLaneFind(img):

    # A1. Apply a distortion correction to raw images
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)
    
    # A1.1 Apply image mask
    leftVertics = np.array([[(150,720),(400,450),(700,450),(700,720)]])
    rightVertics = np.array([[(1250,720),(1000,450),(700,450),(700,720)]],dtype=np.int32)
    
    leftCropped = regionOfInterest(undistorted,leftVertics)
    rightCropped = regionOfInterest(undistorted,rightVertics)
    
    # A2. Use color transforms, gradients, etc., to create a thresholded binary image
    leftCombined = colorTransform(leftCropped)
    rightCombined = colorTransform(rightCropped)
    
    # A3. Apply a perspective transform to rectify binary image ("birds-eye view")
    left_binary_warped = cv2.warpPerspective(leftCombined, M, img_size, flags=cv2.INTER_NEAREST)
    right_binary_warped = cv2.warpPerspective(rightCombined, M, img_size, flags=cv2.INTER_NEAREST)
    
    left_binary_warped = cv2.flip(left_binary_warped, 0)
    right_binary_warped = cv2.flip(right_binary_warped, 0)
    
    # A4. Detect lane pixels and fit to find the lane boundary
    if (leftLine.detected == False or rightLine.detected == False):
        slidingWindows(left_binary_warped, right_binary_warped, leftLine, rightLine, confidenceCheck = 1, polynomialCheck = 1, segmentCheck = 1, debugMode = 1)
    elif (leftLine.detected == True and rightLine.detected == True):
        knownLines(left_binary_warped, right_binary_warped, leftLine, rightLine, confidenceCheck = 1, polynomialCheck = 1, segmentCheck = 1)
    
    if not(leftLine.noLineFound == True and rightLine.noLineFound == True):
        # A5. Determine the curvature of the lane and vehicle position with respect to center
        curvatureCalc(leftLine, rightLine)
        CR = np.around((leftLine.CR + rightLine.CR)/2, decimals = 1)
        text = 'Radius of Curvature = ' + str(CR) + '[m]'
    
        if (leftLine.offcenter > 0):
            text2 = 'Left from center by ' + str(np.around(leftLine.offcenter, decimals = 1)) + '[m]'
        elif (leftLine.offcenter < 0):
            text2 = 'Right from center by ' + str(-np.around(leftLine.offcenter, decimals = 1)) + '[m]'
    
        # A6. Warp the detected lane boundaries back onto the original image
        newwarp = warpBackLines(left_binary_warped, leftLine, rightLine, Minv)
    
        # A7. Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position
        result = cv2.addWeighted(undistorted, 1, newwarp, 0.4, 0)
        result = cv2.putText(result,text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2)
        result = cv2.putText(result,text2, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2)
    
    else:
        warning = 'No lane found, please take control'
        result = cv2.putText(undistorted, warning, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 2)
    return result

## Apply complete filter to the videos

In [13]:
leftLine = Line()
rightLine = Line()
white_output = 'outputs/prcsd_project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(advLaneFind)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video outputs/prcsd_project_video.mp4
[MoviePy] Writing video outputs/prcsd_project_video.mp4


100%|█████████▉| 1260/1261 [03:13<00:00,  6.33it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputs/prcsd_project_video.mp4 

CPU times: user 5min 22s, sys: 31.1 s, total: 5min 53s
Wall time: 3min 14s


In [14]:
# Play the video on notebook
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [15]:
leftLine = Line()
rightLine = Line()
white_output = 'outputs/prcsd_challenge_video.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(advLaneFind)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video outputs/prcsd_challenge_video.mp4
[MoviePy] Writing video outputs/prcsd_challenge_video.mp4


100%|██████████| 485/485 [01:13<00:00,  6.55it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputs/prcsd_challenge_video.mp4 

CPU times: user 2min 2s, sys: 12.2 s, total: 2min 14s
Wall time: 1min 14s


In [16]:
# Play the video on notebook
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [17]:
leftLine = Line()
rightLine = Line()
white_output = 'outputs/prcsd_harder_challenge_video.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(advLaneFind)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video outputs/prcsd_harder_challenge_video.mp4
[MoviePy] Writing video outputs/prcsd_harder_challenge_video.mp4


100%|█████████▉| 1199/1200 [03:29<00:00,  6.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: outputs/prcsd_harder_challenge_video.mp4 

CPU times: user 5min 28s, sys: 39 s, total: 6min 7s
Wall time: 3min 31s


In [19]:
# Play the video on notebook
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))